<a href="https://colab.research.google.com/github/jonathanluo23/NBA-allstar-all-nba-Prediction/blob/main/XGB_Hypertuning_All_NBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Capstone/features_allnba')
data

,PTS,FGM,PIE,FTA,W,PLUS_MINUS,FGA,POSS,FTM,USG_PCT,Y
0,0.331965,0.298372,0.549559,0.309540,0.512548,0.588911,0.409601,0.402178,0.307733,0.477575,0
1,0.331965,0.359273,0.452906,0.358982,0.371560,0.299963,0.364825,0.474800,0.269963,0.335325,0
2,0.663537,0.707735,0.601646,0.625887,0.623642,0.718178,0.733066,0.729872,0.600212,0.715859,0
3,0.621795,0.694274,0.755714,0.615881,0.195063,0.867368,0.649253,0.302438,0.614219,0.488580,0
4,0.762711,0.879514,0.902713,0.635549,0.575695,0.718178,0.859997,0.781053,0.640186,0.768371,0
...,...,...,...,...,...,...,...,...,...,...,...
5622,0.847901,0.874806,0.749689,0.814499,0.635318,0.501347,0.870871,0.859264,0.841262,0.845072,0
5623,0.586318,0.666163,0.347545,0.421983,0.039722,0.000000,0.757987,0.217864,0.497239,0.428994,0
5624,0.449480,0.437141,0.452906,0.474918,0.472608,0.449558,0.438665,0.459714,0.428622,0.382823,0
5625,0.501579,0.523485,0.325719,0.335126,0.680907,0.675272,0.550525,0.660283,0.342032,0.375000,0


In [ ]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

#Hypertuning


In [ ]:
#Initiate the LR model with random hyperparameters
import xgboost as xgb
import warnings
warnings.filterwarnings('ignore')
xgb0 = xgb.XGBClassifier(learning_rate = 0.05, n_estimators=300, max_depth=5)

In [ ]:
xgb0.fit(X,y)

XGBClassifier(learning_rate=0.05, max_depth=5, n_estimators=300)

In [ ]:
xgb0.score(X,y)

1.0

In [ ]:
# You will need the following dependencies for applying Cross-validation and evaluating the cross-validated score

from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

# Build the k-fold cross-validator

kfold = KFold(n_splits=3, random_state=2019,shuffle=True)

In [ ]:
result = cross_val_score(xgb0, X, y, cv=kfold, scoring='accuracy')
print('accuracy', result.mean())
result = cross_val_score(xgb0, X, y, cv=kfold, scoring='precision')
print('precision', result.mean())
result = cross_val_score(xgb0, X, y, cv=kfold, scoring='recall')
print('recall', result.mean())

accuracy 0.9863158493248045
precision 0.7949882868829171
recall 0.7226227795193313


In [ ]:
learning_rate = [0.1,0.01,0.05]
n_estimators = [200,300,400]
max_depth = [3,5,10]
gamma =[5,6,7]
colsample_bytree=[1.0, 1.5]
param_grid = dict(learning_rate=learning_rate,n_estimators=n_estimators,max_depth=max_depth, gamma=gamma, colsample_bytree=colsample_bytree)

In [ ]:
import numpy as np
np.random.seed(7)
n_estimators1 = np.random.randint(200,300,400)
max_depth1 = np.random.randint(3,5,10)
gamma1 = np.random.randint(5,6,7)
param_grid1 = dict(n_estimators=n_estimators1,max_depth=max_depth1, gamma=gamma1)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
random = RandomizedSearchCV(estimator=xgb0, param_distributions=param_grid1, cv = 3, n_jobs=-1)

import time
start_time = time.time()
random_result = random.fit(X, y)
# Summarize results
print("Best: %f using %s" % (random_result.best_score_, random_result.best_params_))
print("Execution time: " + str((time.time() - start_time)) + ' ms')

Best: 0.987916 using {'n_estimators': 219, 'max_depth': 3, 'gamma': 5}
Execution time: 15.592237949371338 ms


#Evaluation


In [ ]:
# balance the data
from imblearn.over_sampling import SMOTE

from sklearn import model_selection

In [ ]:
# resample/balance the data
sm = SMOTE(random_state = 2021) 
X_res, y_res = sm.fit_resample(X, y) 

In [ ]:
clf = xgb.XGBClassifier(learning_rate = 0.05,n_estimators= 219, max_depth= 3, gamma= 5)

In [ ]:
def my_eval(X, y, classifer = clf, k=10, scoring = 'f1'):
  '''
  return evaluation results (f1-score or ROC_AUC). 
  Built in k-fold evaluation.
  INPUTS:
  ----
  - X: features; DataFrame or Numpy ndarray;
  - y: target; DataFrame or Numpy ndarray;
  - classifier: any sklearn (or its add-on) based classifier
  - k: number of folds in cross validation
  - scoring: evaluation metric ('f1' default or 'roc_auc')
  OUTPUT:
  ----
  bias/variance score of selected metric.
  '''
  scores = []
  for i in range(50):
    #### generate random numbers to shuffle the data for training and test
    np.random.seed(2021)
    random_int = np.random.randint(0,3000)
    #### create cross validation folds
    kfold = model_selection.KFold(n_splits=k, random_state=random_int, shuffle=True)
    #### record the score
    score = model_selection.cross_val_score(clf, X=X, y=y, cv=kfold, scoring=scoring)
    scores.append(score)
  scores = np.array(scores)
  #### we need to calculate the bias (average score) and viariance (std)
  bias, variance = round(scores.mean(),4), round(scores.std(),4)
  return(bias, variance)

In [ ]:
# getting averaged f1_score from 10-fold CV (default)
my_eval(X_res, y_res, clf, 10)

(0.9883, 0.003)

In [ ]:
# getting averaged ROC_AUC from 10-fold CV
my_eval(X_res, y_res, clf, 10, 'roc_auc')

(0.9982, 0.0008)

In [ ]:
clf = xgb.XGBClassifier(n_estimators= 219, max_depth= 3, gamma= 5)

In [ ]:
# getting averaged f1_score from 10-fold CV (default)
my_eval(X_res, y_res, clf, 10)

(0.9881, 0.0031)

In [ ]:
# getting averaged ROC_AUC from 10-fold CV
my_eval(X_res, y_res, clf, 10, 'roc_auc')

(0.9982, 0.0008)